<a href="https://colab.research.google.com/github/adithya-ananth/Star-Pattern-Recognition/blob/main/Constellation_Coordinates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the necessary modules
from bs4 import BeautifulSoup as bs
import requests

from urllib.parse import unquote

import math
from astropy import units as u
from astropy.coordinates import SkyCoord, get_constellation

!pip install astroquery
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad

In [ ]:
def find_stars(constellation_name):
  stars = []

  r = requests.get(f'https://en.wikipedia.org/wiki/List_of_stars_in_{constellation_name}')
  soup = bs(r.content, features = 'html.parser')

  # access tables
  tables = soup.find_all("tbody")

  for table in tables:
    # access each row
    rows = table.find_all("tr")

    for row in rows:
      star_name = row.find("td")

      try:
        req_name = star_name.find('a')['href']
        req_name = req_name.replace('_', ' ')

        # if the constellation is Bootes, decode the unicoded star name
        if constellation_name == "Bootes":
          req_name = unquote(req_name)
          req_name = req_name.replace('ö', 'o')

        # if the wiki link of the star has '(star)' in it
        if req_name.find('(star)') > 0:
          req_name = req_name[0:-7]

        if req_name[0:6] == '/wiki/':
          stars.append(req_name[6:])
        else:
          stars.append(star_name.text)
      except:
        pass

  return stars

In [ ]:
# Get ra and dec values

def get_star_data(star_name):
    # get result table
    result_table = Simbad.query_object(star_name)

    # get ra and dec values and convert them
    ra_converted = float(result_table[0][1][:2])*15 + float(result_table[0][1][3:5])/4 + float(result_table[0][1][6:])*(25/6000)
    dec_converted = float(result_table[0][2][1:3]) + float(result_table[0][2][4:6])/60 + float(result_table[0][2][7:])/3600

    if result_table[0][2][0] == '-':
      dec_converted = -dec_converted

    #print(ra_converted, dec_converted)
    return SkyCoord(ra=round(ra_converted,2)*u.deg, dec=round(dec_converted,4)*u.deg)

In [ ]:
# Define a list of 88 constellations (for reference)
constellations = [
        "Aquila", "Andromeda", "Sculptor", "Ara", "Libra", "Cetus", "Aries",
        "Scutum", "Pyxis", "Bootes", "Caelum", "Chamaeleon", "Cancer", "Capricornus", "Carina", "Cassiopeia",
        "Centaurus", "Cepheus", "Coma Berenices", "Canes Venatici", "Auriga", "Columba", "Circinus", "Crater",
        "Corona Australis", "Corona Borealis", "Corvus", "Crux", "Cygnus", "Delphinus", "Dorado", "Draco",
        "Norma", "Eridanus", "Sagitta", "Fornax", "Gemini", "Camelopardalis", "Canis Major", "Ursa Major",
        "Grus", "Hercules", "Horologium", "Hydra", "Hydrus", "Indus", "Lacerta", "Monoceros", "Lepus", "Leo",
        "Lupus", "Lynx", "Lyra", "Antlia", "Microscopium", "Musca", "Octans", "Apus", "Ophiuchus", "Orion",
        "Pavo", "Pegasus", "Pictor", "Perseus", "Equuleus", "Canis Minor", "Leo Minor", "Vulpecula",
        "Ursa Minor", "Phoenix", "Pisces", "Piscis Austrinus", "Volans", "Puppis", "Reticulum", "Sagittarius",
        "Scorpius", "Scutum", "Serpens", "Sextans", "Mensa", "Taurus", "Telescopium", "Tucana", "Triangulum",
        "Triangulum Australe", "Aquarius", "Virgo", "Vela"
    ]

# Create a list of lists, where each list contains stars obtained for each constellaition (sorted in order of decreasing brightness)
stars_list = []
for constellation in constellations:
  stars_list.append(find_stars(constellation))

Main Code:

In [ ]:
# Loop over the stars and print their coordinates
table = {}
for_df = {}
for_df['Constellation'] = []
for_df['Star Name'] = []
for_df['x coordinate'] = []
for_df['y coordinate'] = []
for stars in stars_list:
    # Get the coordinate object of the first (i.e. brightest) star
    # If the first star isn't available in astropy's database, we choose the i'th star as our reference
    i = 0
    while i > -1:
      try:
        coord_one = SkyCoord.from_name(stars[i])
        break
      except:
        i = i+1

    # Get the constellation of the first star
    constellation = get_constellation(coord_one)
    print(f"{constellation}: ")
    if not constellation in table:
        table[constellation] = []

    # Define the reference frame and origin
    frame = 'icrs'
    origin = get_star_data(stars[i])

    # Get top 20 brightest stars
    for star in stars[0:20]:
      try:
        # Calculate the relative coordinates to the origin
        coord = SkyCoord.from_name(star)
        dx = (coord.ra - origin.ra).to(u.deg).value
        dy = (coord.dec - origin.dec).to(u.deg).value

        # Print the star name and coordinates
        print(f"{star} : ({dx:.2f}, {dy:.2f})")
        table[constellation].append({star: [dx.__round__(2), dy.__round__(2)]})
        for_df['Constellation'].append(constellation)
        for_df['Star Name'].append(star)
        for_df['x coordinate'].append(dx.__round__(2))
        for_df['y coordinate'].append(dy.__round__(2))
      except:
        pass




    print()

import pandas as pd, json

# Convert dictionary to JSON string
json_string = json.dumps(table)

# Save JSON string to a file
with open('data.json', 'w') as f:
    f.write(json_string)
    f.close()

df = pd.DataFrame(for_df)
df.to_csv('star_coordinates1.csv')

Aquila: 
Altair : (-0.00, 0.00)
Gamma Aquilae : (-1.14, 1.74)
Zeta Aquilae : (-11.35, 5.00)
Theta Aquilae : (5.13, -9.69)
Delta Aquilae : (-6.33, -5.75)
Lambda Aquilae : (-11.14, -13.75)
Beta Aquilae : (1.13, -2.46)
Eta Aquilae : (0.42, -7.86)
Epsilon Aquilae : (-12.79, 6.20)
12 Aquilae : (-12.28, -14.61)
71 Aquilae : (11.88, -9.97)
Iota Aquilae : (-3.52, -10.15)
Mu Aquilae : (-2.15, 4.47)
Nu Aquilae : (-1.18, 2.89)
Xi Aquilae : (0.86, -0.41)
69 Aquilae : (9.71, -11.75)
70 Aquilae : (11.48, -11.42)
Kappa Aquilae : (-3.48, -15.90)
26 Aquilae : (-7.56, -14.28)
4 Aquilae : (-16.49, -6.81)

Andromeda: 
Alpheratz : (-0.00, 0.00)
Mirach : (15.33, 6.53)
Gamma Andromedae : (28.87, 13.24)
Delta Andromedae : (7.73, 1.77)
Andromeda Galaxy : (8.58, 12.18)
51 Andromedae : (22.40, 19.54)
Omicron Andromedae : (343.38, 13.24)
Lambda Andromedae : (352.29, 17.37)
Mu Andromedae : (35.26, 10.44)
Zeta Andromedae : (9.73, -4.82)
Upsilon Andromedae : (22.10, 12.32)
Kappa Andromedae : (353.00, 15.24)
Phi Andr

In [ ]:
# # Convert dictionary to JSON string
# json_string = json.dumps(data)

# Save JSON string to a file
with open('data.json', 'r') as f:
    k = f.readlines()
    f.close()
print(*k)

df.sample(20)


{"Aquila": [{"Altair": [-0.0, 0.0]}, {"Gamma Aquilae": [-1.14, 1.74]}, {"Zeta Aquilae": [-11.35, 5.0]}, {"Theta Aquilae": [5.13, -9.69]}, {"Delta Aquilae": [-6.33, -5.75]}, {"Lambda Aquilae": [-11.14, -13.75]}, {"Beta Aquilae": [1.13, -2.46]}, {"Eta Aquilae": [0.42, -7.86]}, {"Epsilon Aquilae": [-12.79, 6.2]}, {"12 Aquilae": [-12.28, -14.61]}, {"71 Aquilae": [11.88, -9.97]}, {"Iota Aquilae": [-3.52, -10.15]}, {"Mu Aquilae": [-2.15, 4.47]}, {"Nu Aquilae": [-1.18, 2.89]}, {"Xi Aquilae": [0.86, -0.41]}, {"69 Aquilae": [9.71, -11.75]}, {"70 Aquilae": [11.48, -11.42]}, {"Kappa Aquilae": [-3.48, -15.9]}, {"26 Aquilae": [-7.56, -14.28]}, {"4 Aquilae": [-16.49, -6.81]}], "Andromeda": [{"Alpheratz": [-0.0, 0.0]}, {"Mirach": [15.33, 6.53]}, {"Gamma Andromedae": [28.87, 13.24]}, {"Delta Andromedae": [7.73, 1.77]}, {"Andromeda Galaxy": [8.58, 12.18]}, {"51 Andromedae": [22.4, 19.54]}, {"Omicron Andromedae": [343.38, 13.24]}, {"Lambda Andromedae": [352.29, 17.37]}, {"Mu Andromedae": [35.26, 10.44]}

,Constellation,Star Name,x coordinate,y coordinate
1351,Pisces,Omicron Piscium,3.48,-6.19
914,Monoceros,3 Monocerotis,-24.85,-1.05
689,Fornax,Lambda2 Fornacis,-8.78,-5.59
685,Fornax,AI For,1.88,4.86
688,Fornax,Iota1 Fornacis,-8.98,-1.06
5,Aquila,Lambda Aquilae,-11.14,-13.75
362,Coma Berenices,13 Comae Berenices,-11.89,-1.78
901,Lacerta,HD 216174,4.62,5.62
1294,Vulpecula,1 Vulpeculae,-3.13,-3.27
1035,Antlia,Zeta1 Antliae,-14.10,-0.82


**Change of Basis (for the coordinates)**

In [ ]:
import numpy as np, matplotlib.pyplot as plt, random as rd

coordinates = np.array([[rd.randint(1,10) for j in range(2)] for i in range(10)])
print(coordinates, '\n')

new_basis = np.array([
    [0.5,0],
    [0,1]
])

new_coordinates = np.matmul(coordinates, np.linalg.inv(new_basis))

print(new_coordinates)


[[ 9  5]
 [ 7  3]
 [ 7  4]
 [ 8 10]
 [ 8  7]
 [ 4 10]
 [ 6  4]
 [ 5 10]
 [ 1  8]
 [ 7  3]] 

[[18.  5.]
 [14.  3.]
 [14.  4.]
 [16. 10.]
 [16.  7.]
 [ 8. 10.]
 [12.  4.]
 [10. 10.]
 [ 2.  8.]
 [14.  3.]]


**Old Code Starts Here**

In [ ]:
# Import the necessary modules
from astropy import units as u
from astropy.coordinates import SkyCoord, get_constellation

# Define a list of 88 constellations (for reference)
constellations = [
        "Aquila", "Andromeda", "Sculptor", "Ara", "Libra", "Cetus", "Aries",
        "Scutum", "Pyxis", "Bootes", "Caelum", "Chamaeleon", "Cancer", "Capricornus", "Carina", "Cassiopeia",
        "Centaurus", "Cepheus", "Coma Berenices", "Canes Venatici", "Auriga", "Columba", "Circinus", "Crater",
        "Corona Australis", "Corona Borealis", "Corvus", "Crux", "Cygnus", "Delphinus", "Dorado", "Draco",
        "Norma", "Eridanus", "Sagitta", "Fornax", "Gemini", "Camelopardalis", "Canis Major", "Ursa Major",
        "Grus", "Hercules", "Horologium", "Hydra", "Hydrus", "Indus", "Lacerta", "Monoceros", "Lepus", "Leo",
        "Lupus", "Lynx", "Lyra", "Antlia", "Microscopium", "Musca", "Octans", "Apus", "Ophiuchus", "Orion",
        "Pavo", "Pegasus", "Pictor", "Perseus", "Equuleus", "Canis Minor", "Leo Minor", "Vulpecula",
        "Ursa Minor", "Phoenix", "Pisces", "Piscis Austrinus", "Volans", "Puppis", "Reticulum", "Sagittarius",
        "Scorpius", "Scutum", "Serpens", "Sextans", "Mensa", "Taurus", "Telescopium", "Tucana", "Triangulum",
        "Triangulum Australe", "Aquarius", "Virgo", "Vela"
    ]

'''
This code is no longer required

# Define a list of list of stars for each of the 88 constellations
stars_list = [['Altair','Alshain','Tarazed','Epsilon Aquilae','Zeta Aquilae','Eta Aquilae','Theta Aquilae','Iota Aquilae'],
              ['Alpheratz','Mirach','Almach','Adhil','Segin','Nembus','Angetenar','Tau Andromedae','Delta Andromedae','Upsilon Andromedae','Zeta Andromedae','Rho Andromedae','Pi Andromedae','Iota Andromedae','Lambda Andromedae','Kappa Andromedae'],
              ['Alpha Sculptoris','Beta Sculptoris','Gamma Sculptoris','Delta Sculptoris','Epsilon Sculptoris'],
              ['Alpha Arae','Beta Arae','Zeta Arae','Gamma Arae','Delta Arae','Theta Arae','Eta Arae'],
              ['Alpha Librae','Beta Librae','Gamma Librae','Delta Librae','Sigma Librae','Mu Librae','Nu Librae','42 Librae'],
              ['Alpha Ceti','Beta Ceti','Gamma Ceti','Delta Ceti','Epsilon Ceti','Zeta Ceti','Eta Ceti','Theta Ceti','Iota Ceti','Kappa Ceti','Lambda Ceti','Omicron Ceti'], # requires 15 but only 13 stars work
              ['Hamal','Sheratan','Mesarthim','Botein','Alderamin']
             ]
'''

# Define the coordinates of the brightest star of each constellation
coordinates = [SkyCoord(ra=297.70*u.deg, dec=8.8683*u.deg),
               SkyCoord(ra=2.10*u.deg, dec=29.0906*u.deg),
               SkyCoord(ra=14.65*u.deg, dec=-29.3574*u.deg),
               SkyCoord(ra=262.96*u.deg, dec=-49.8761*u.deg),
               SkyCoord(ra=222.72*u.deg, dec=-16.0416*u.deg)
              ]

# Loop over the stars and print their coordinates
for stars in stars_list:
    # Get the coordinate object of the first star
    coord_one = SkyCoord.from_name(stars[0])
    # Get the constellation of the first star
    constellation = get_constellation(coord_one)
    print(f"{constellation}: ")

    # Define the reference frame and origin
    frame = 'icrs'
    origin = coordinates[stars_list.index(stars)]

    for star in stars:
      # Calculate the relative coordinates to the origin
      coord = SkyCoord.from_name(star)
      dx = (coord.ra - origin.ra).to(u.deg).value
      dy = (coord.dec - origin.dec).to(u.deg).value

      # Print the star name and coordinates
      print(f"{star} : ({dx:.2f}, {dy:.2f})")

    print()

Aquila: 
Altair : (-0.00, 0.00)
Alshain : (1.13, -2.46)
Tarazed : (-1.14, 1.74)
Epsilon Aquilae : (-12.79, 6.20)
Zeta Aquilae : (-11.35, 5.00)
Eta Aquilae : (0.42, -7.86)
Theta Aquilae : (5.13, -9.69)
Iota Aquilae : (-3.52, -10.15)

Andromeda: 
Alpheratz : (-0.00, -0.00)
Mirach : (15.33, 6.53)
Almach : (28.87, 13.24)
Adhil : (18.49, 16.44)
Segin : (26.50, 34.58)


KeyboardInterrupt: 

In [ ]:
# Import the necessary modules
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
# !pip install astroquery
from astroquery.gaia import Gaia

# Set the row limit for returned data
Gaia.ROW_LIMIT = 10000

# Define a function to get the coordinates of the brightest star in a constellation
def get_brightest_star(constellation):
    # Query the Gaia catalog for stars in the given constellation
    query = f"SELECT TOP 1 * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',6.0,0.0,180.0))=1 AND phot_g_mean_mag IS NOT NULL AND parallax IS NOT NULL AND constellation_name = '{constellation}' ORDER BY phot_g_mean_mag ASC"
    job = Gaia.launch_job_async(query)
    table = job.get_results()

    # Extract the relevant columns from the table
    ra = table['ra'][0] * u.deg # Right ascension
    dec = table['dec'][0] * u.deg # Declination
    plx = table['parallax'][0] * u.mas # Parallax
    gmag = table['phot_g_mean_mag'][0] # G-band magnitude

    # Create a SkyCoord object for the star
    star = SkyCoord(ra=ra, dec=dec, distance=Distance(parallax=plx), frame='icrs')

    # Return the star and its magnitude
    return star, gmag

# Define a function to get the coordinates of all the major stars in a constellation
def get_major_stars(constellation):
    # Query the Gaia catalog for stars in the given constellation with magnitude less than 6
    query = f"SELECT * FROM gaiadr2.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),CIRCLE('ICRS',6.0,0.0,180.0))=1 AND phot_g_mean_mag IS NOT NULL AND parallax IS NOT NULL AND constellation_name = '{constellation}' AND phot_g_mean_mag < 6 ORDER BY phot_g_mean_mag ASC"
    job = Gaia.launch_job_async(query)
    table = job.get_results()

    # Extract the relevant columns from the table
    ra = table['ra'] * u.deg # Right ascension
    dec = table['dec'] * u.deg # Declination
    plx = table['parallax'] * u.mas # Parallax
    gmag = table['phot_g_mean_mag'] # G-band magnitude

    # Create a SkyCoord object for the stars
    stars = SkyCoord(ra=ra, dec=dec, distance=Distance(parallax=plx), frame='icrs')

    # Return the stars and their magnitudes
    return stars, gmag

# Define a function to print the coordinates of all the major stars in a constellation
def print_coordinates(constellation):
    # Get the brightest star and its magnitude
    brightest_star, brightest_mag = get_brightest_star(constellation)

    # Get the major stars and their magnitudes
    major_stars, major_mags = get_major_stars(constellation)

    # Print the name of the constellation and the number of major stars
    print(f"Constellation: {constellation}")
    print(f"Number of major stars: {len(major_stars)}")

    # Print the coordinates of the brightest star as the origin
    print(f"Brightest star: (0, 0, 0), magnitude = {brightest_mag:.2f}")

    # Loop over the major stars
    for i, star in enumerate(major_stars):
        # Skip the brightest star
        if star == brightest_star:
            continue

        # Calculate the relative coordinates of the star with respect to the brightest star
        dx = star.cartesian.x - brightest_star.cartesian.x
        dy = star.cartesian.y - brightest_star.cartesian.y
        dz = star.cartesian.z - brightest_star.cartesian.z

        # Print the coordinates and the magnitude of the star
        print(f"Star {i+1}: ({dx.value:.2f}, {dy.value:.2f}, {dz.value:.2f}), magnitude = {major_mags[i]:.2f}")

    # Print a blank line
    print()

# Define a list of all 88 constellations
constellations = ['Andromeda', 'Antlia', 'Apus', 'Aquarius', 'Aquila', 'Ara', 'Aries', 'Auriga', 'Bootes', 'Caelum', 'Camelopardalis', 'Cancer', 'Canes Venatici', 'Canis Major', 'Canis Minor', 'Capricornus', 'Carina', 'Cassiopeia', 'Centaurus', 'Cepheus', 'Cetus', 'Chamaeleon', 'Circinus', 'Columba', 'Coma Berenices', 'Corona Australis', 'Corona Borealis', 'Corvus', 'Crater', 'Crux', 'Cygnus', 'Delphinus', 'Dorado', 'Draco', 'Equuleus', 'Eridanus', 'Fornax', 'Gemini', 'Grus', 'Hercules', 'Horologium', 'Hydra', 'Hydrus', 'Indus', 'Lacerta', 'Leo', 'Leo Minor', 'Lepus', 'Libra', 'Lupus', 'Lynx', 'Lyra', 'Mensa', 'Microscopium', 'Monoceros', 'Musca', 'Norma', 'Octans', 'Ophiuchus', 'Orion', 'Pavo', 'Pegasus', 'Perseus', 'Phoenix', 'Pictor', 'Pisces', 'Piscis Austrinus', 'Puppis', 'Pyxis', 'Reticulum', 'Sagitta', 'Sagittarius', 'Scorpius', 'Sculptor', 'Scutum', 'Serpens', 'Sextans', 'Taurus', 'Telescopium', 'Triangulum', 'Triangulum Australe', 'Tucana', 'Ursa Major', 'Ursa Minor', 'Vela', 'Virgo', 'Volans', 'Vulpecula']

# Loop over the constellations and print the coordinates of their major stars
for constellation in constellations:
    print(constellation)
    print_coordinates(constellation)


Andromeda
500 Error 500:
null


HTTPError: Error 500:
null

In [ ]:
!pip install astroquery
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad

def get_star_data(star_name):
    result_table = Simbad.query_object(star_name)
    print(result_table)

get_star_data("Hamal")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.2/910.2 kB 25.0 MB/s eta 0:00:00
 MAIN_ID        RA           DEC      ...     COO_BIBCODE     SCRIPT_NUMBER_ID
             "h:m:s"       "d:m:s"    ...                                     
--------- ------------- ------------- ... ------------------- ----------------
* alf Ari 02 07 10.4057 +23 27 44.703 ... 2007A&A...474..653V                1
